In [1]:
import torch
from data_loader import CustomDataLoader
from torch.utils.data import DataLoader
import ukis_metrics.seg_metrics as segm

import warnings
warnings.filterwarnings('ignore')

data_path = '../data_prepped'
image_path = data_path + '/{}/img/*'
mask_path = data_path + '/{}/msk/*'
 
# Use custom data loaders for S1S2 dataset
test_dataset = CustomDataLoader(image_path.format('val'), mask_path.format('val'))
test_loader = DataLoader(dataset = test_dataset, batch_size = 8, shuffle = False, num_workers = 8)

In [2]:
from model import SegmentationModel
model_ckpt = './networks/2024_04_11-13_17_35_encoder_efficientnet-b0_weights_imagenet_epochs_100_batchsize_16_lr_0.001/model_0.40356_val_avg.pt'
model = SegmentationModel('efficientnet-b0', 'imagenet')
model.load_state_dict(torch.load(model_ckpt))
model.eval()

SegmentationModel(
  (cnn): Unet(
    (encoder): EfficientNetEncoder(
      (_conv_stem): Conv2dStaticSamePadding(
        3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
        (static_padding): ZeroPad2d((0, 1, 0, 1))
      )
      (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_blocks): ModuleList(
        (0): MBConvBlock(
          (_depthwise_conv): Conv2dStaticSamePadding(
            32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
            (static_padding): ZeroPad2d((1, 1, 1, 1))
          )
          (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (_se_reduce): Conv2dStaticSamePadding(
            32, 8, kernel_size=(1, 1), stride=(1, 1)
            (static_padding): Identity()
          )
          (_se_expand): Conv2dStaticSamePadding(
            8, 32, kernel_size=(1, 1), stride=(1, 1)
            (static_padding): I

In [3]:
def metric_for_ratio(masks, pred_masks, ratio):
    pred_masks = (pred_masks > ratio).cpu().bool()
    masks = masks.bool()

    tpfptnfn = segm.tpfptnfn(masks == 1, pred_masks, None)
    metrics = segm.segmentation_metrics(tpfptnfn)
    # print(ratio, metrics)
    return metrics

In [4]:
torch.multiprocessing.set_sharing_strategy('file_system')
from tqdm import tqdm 
device = 'cuda'
model = model.to(device)


y_true, y_pred = [], []

i = 0
with torch.no_grad():
    for images, masks in tqdm(test_loader):
        images = images.to(device, dtype = torch.float32)

        logits = model(images, None)
        preds = torch.sigmoid(logits).cpu()
        
        y_pred.append(preds)
        y_true.append(masks.bool())

        # Tested different ratios to find the best one for performance
        # for ratio in [0, 0.05, 0.1, 0.2, 0.4, 0.8, 0.85, 0.9, 0.95]: 
        #     metric_for_ratio(masks, pred_masks, ratio)
        
        # ratio = 0.8
        # metric_for_ratio(masks, pred_masks, ratio)
        # i += 1
        # if i == 10:
        #     break

100%|██████████████████████████████████████████████████████████████████████████████████████████| 3107/3107 [46:33<00:00,  1.11it/s]


In [5]:
y_pred = torch.cat(y_pred)
y_true = torch.cat(y_true)
images.shape

torch.Size([6, 3, 512, 512])

In [ ]:
ratio = 0.8
metric_for_ratio(y_true, y_pred, ratio)

In [9]:
import numpy as np
ious = 0
shape = np.prod(y_true.shape[1:])
for i in tqdm(range(y_true.shape[0])):
    m = metric_for_ratio(y_true[i], y_pred[i], ratio)
    ious += m['iou']*shape #np.prod(y_true.shape[1:])

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [01:56<00:00,  2.91s/it]


In [17]:
y_true.bool()

tensor([[[[False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False]]],


        [[[False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False]]],


        [[[False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [False, False, False,  ..., False, Fa

In [10]:
y_true.shape

torch.Size([40, 1, 512, 512])

In [12]:
ious/np.prod(y_true.shape)#.shape[0]

0.9194150000000001

In [6]:
import pickle

path = './networks/2024_04_16-16_28_15_encoder_efficientnet-b0_weights_imagenet_epochs_100_batchsize_16_lr_0.001/params.pkl'
with open(path, 'rb') as f:
    loaded_dict = pickle.load(f)
loaded_dict

FileNotFoundError: [Errno 2] No such file or directory: './networks/2024_04_16-16_28_15_encoder_efficientnet-b0_weights_imagenet_epochs_100_batchsize_16_lr_0.001/params.pkl'

In [ ]:
import ast
ast.literal_eval(str(loaded_dict))